## IMPORTS

In [1]:
import numpy as np
import scipy.stats as ss
import math, itertools
import random
import time
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp as OR
from gurobipy import *

### METHODS FOR GENERATION

In [ ]:
# sample from a discretized gaussian distribution truncated to [lb,ub] 
def DiscreteBetaRand(mean):
    a = 2.5
    b = 10
    m = a/(a+b)
    lb=1
    ub=(round(mean/m))+1
    x = np.arange(lb,ub+1)
    xU, xL = x + 0.5, x - 0.5 
    prob = ss.beta.cdf(xU, a,b,0,ub-lb) - ss.beta.cdf(xL, a,b,0,ub-lb)
    prob = prob / prob.sum() #normalize the probabilities so their sum is 1
    return np.random.choice(x, p = prob)

In [ ]:
# create a perfect bin packing input. Output problem instance and solution
def PerfectBinPacking(mean,binCount,capacity):
    TYPES = [] # set of item types (volumes)
    count = {} # count of each item type
    BINS = range(binCount) # set of bins
    pack = {} # encodes optimal solution
    for b in BINS:
        pack.update({b:[]})
    for b in BINS:
        PPB = DiscreteBetaRand(mean) # pieces in this bin
        pos = list(range(1,capacity)) # list of positions
        split = random.sample(pos,PPB-1) # split up capacity 
        split = sorted(split) # sort
        split.append(capacity)
        lastPos = 0
        for i in split:
            item = i - lastPos
            if item in TYPES:
                count[item] += 1
            else:
                TYPES.append(item)
                count.update({item : 1})
            pack[b].append(item)
            lastPos = i       
    return TYPES, count, pack

In [ ]:
# show histogram of PPB (Pieces Per Bin)
def HistoPPB(pack):
    PPB = [] 
    for b in pack.keys():
        PPB.append(len(pack[b]))
    plt.hist(PPB,
         bins=max(PPB)-min(PPB),
         density=False,
         histtype='bar',
         color='b',
         edgecolor='k',
         alpha=0.5)
    plt.xlabel('Pieces Per Bin')
    plt.ylabel('Number of Bins')
    plt.title('Histogram of Pieces Per Bin')
    plt.show()

In [ ]:
# show histogram of item volumes
def HistoVol(ITEMS):
    plt.hist(ITEMS,
         bins=max(ITEMS)-min(ITEMS),
         density=False,
         histtype='bar',
         color='b',
         edgecolor='k',
         alpha=0.5)
    plt.xlabel('Item Volume')
    plt.ylabel('Number of Items')
    plt.title('Histogram of Item Volumes')
    plt.show()

### BIN PACKING ABSTRACT

In [ ]:
# a model for a bin packing problem
def PackBins(TYPES,count,capacity,solver,timeLimit=None,relativeGap=None):
    
    BINS = list(range(sum(count.values()))) # upper bound for bins
    PAIRS = list(itertools.product(TYPES, BINS))
     
    # define model
    m = OR.Solver('packBins', solver)
    param = OR.MPSolverParameters()
    if timeLimit is not None:
        print(m.SetSolverSpecificParametersAsString('''TimeLimit 3000 \n 
                                                       NodeLimit 100 \n
                                                       MIPGap 0.05
                                                     '''))
        #m.SetTimeLimit(timeLimit*1000)
    if relativeGap is not None:
        param.SetDoubleParam(param.RELATIVE_MIP_GAP, relativeGap)
    #m.EnableOutput()
    
    ## CHANGE TO IntVar() FOR IP AND NumVar() FOR LP RELAXATION
    
    # decision variables
    x = {} # number of item type i placed in bin j
    for i,j in PAIRS:
        x[i,j] = m.IntVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
    y = {} # 1 if bin i is used; 0 otherwise
    for i in BINS:
        y[i] = m.IntVar(0, 1, ('%s' % (i)))
        
    # set hint
    init = {}
    for i,j in PAIRS:
        init.update({x[i,j] : 0})
    for i in BINS:
        init.update({y[i] : 0}) 
    j = 0
    for i in TYPES:
        for n in range(count[i]):
            init.update({x[i,j] : 1})
            init.update({y[j] : 1})
            j += 1 
    m.SetHint(init.keys(),init.values())
        
    # objective function
    m.Minimize(sum(y[i] for i in BINS)) # set objective
    
    # subject to: bin capacity
    for j in BINS:
        m.Add(sum(x[i,j]*i for i in TYPES) <= capacity)
        
    # subject to: all items packed
    for i in TYPES:
        m.Add(sum(x[i,j] for j in BINS) == count[i])
        
    # subject to: bin used or not
    for j in BINS:
        M = capacity
        m.Add(sum(x[i,j]*i for i in TYPES) <= M*y[j])
    
    m.Solve(param)
    
    # abs(best_bound - incumbent) / abs(incumbent) [Gurobi]
    incumbent = m.Objective().Value()
    best_bound = m.Objective().BestBound()
    if not incumbent == 0:
        gap = abs(best_bound - incumbent) / abs(incumbent)
    else:
        gap = -1
    print(m.nodes())
    return m.Objective().Value(), gap, m.NumVariables(), m.NumConstraints()

### TESTS

In [ ]:
def Test(mean,binCount,capacity,solver):
    s = time.time()
    TYPES, count, pack = PerfectBinPacking(mean,binCount,capacity)
    opt, var, con = PackBins(TYPES,count,capacity,solver)
    chk = (round(opt) == binCount)
    f = time.time()
    return f-s, chk, var, con

In [ ]:
# testing different solver's solve time
times = {}
truth = []
for binCount in [5,10,15,20,25,30,40,50,75,100,125,150,200]:
    for capacity in [100,150,200,250,300]:
        t, chk, var, con= Test(20,binCount,capacity,OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
        print('('+str(binCount)+', '+str(capacity)+', '+str(t)+', '+str(chk)+', '+str(var)+', '+str(con)+')')
        truth.append(chk)
        times.update({(binCount, capacity, var, con) : (t)})

In [ ]:
# testing optimality gap
times = {}
for binCount in [5,10,15,20,25,30,40,50,75,100,125,150,200]:
    for capacity in [100,150,200,250,300]:
        TYPES, count, pack = PerfectBinPacking(20,binCount,capacity) # generate random problem
        s = time.time() # start clock
        #optG, gapG, varG, conG = PackBins(TYPES,count,capacity,OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
        #t = round((f-s)*20) # give CBC 20 times time Gurobi took
        optCBC, gapCBC, varCBC, conCBC = PackBins(TYPES,count,capacity,OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING,relativeGap=0.05,timeLimit=1)
        f = time.time() # stop clock
        t = f - s
        #print('('+str(binCount)+', '+str(capacity)+', '+str(varCBC)+', '+str(conCBC)+', '+str(gapCBC)+', '+str(t)+')')
        times.update({(binCount, capacity, varCBC, conCBC, gapCBC) : (t)})     

In [ ]:
times = {}
a = [(5, 100, 1936, 257, 0.0, 0.4729311466217041),
(5, 150, 2990, 282, 0.0, 0.649367094039917),
(5, 200, 2176, 167, 0.0, 0.15649724006652832),
(5, 250, 3296, 237, 0.0, 0.6368992328643799),
(5, 300, 3094, 215, 0.0, 0.7364311218261719),
(10, 100, 4598, 439, 0.0, 1.8920080661773682),
(10, 150, 7364, 553, 0.0, 4.207428216934204),
(10, 200, 6510, 406, 0.0, 3.02231764793396),
(10, 250, 9424, 533, 0.0, 8.665142059326172),
(10, 300, 7436, 337, 0.0, 6.516972064971924),
(15, 100, 7101, 552, 0.0, 5.730154991149902),
(15, 150, 8547, 498, 0.0, 4.4572741985321045),
(15, 200, 11046, 567, 0.0, 11.472437143325806),
(15, 250, 15141, 666, 0.0, 21.198025941848755),
(15, 300, 16043, 586, 0.0, 19.105751752853394),
(20, 100, 11100, 769, 0.04761904761904745, 0.980297327041626),
(20, 150, 16776, 967, 0.04761904761904779, 2.4003589153289795),
(20, 200, 17650, 755, 0.047619047619047616, 21.70271110534668),
(20, 250, 20760, 751, 0.04761904761904745, 2.1917920112609863),
(20, 300, 20995, 710, 0.04761904761904745, 2.540472984313965),
(25, 100, 15496, 1217, 0.0, 12.693669080734253),
(25, 150, 22116, 1201, 0.038461538461538464, 21.359671115875244),
(25, 200, 23691, 946, 0.038461538461538325, 22.203800201416016),
(25, 250, 29526, 1092, 0.038461538461538596, 51.9524929523468),
(25, 300, 27150, 798, 0.038461538461538325, 43.7290301322937),
(30, 100, 21747, 1350, 0.0, 25.304680109024048),
(30, 150, 24288, 1147, 0.03225806451612903, 68.24879884719849),
(30, 200, 33060, 1197, 0.03225806451612903, 53.24610900878906),
(30, 250, 38709, 1190, 0.03225806451612926, 65.31072497367859),
(30, 300, 39463, 1244, 0.03225806451612892, 59.65176701545715),
(40, 100, 25513, 1676, 0.0243902439024392, 53.21937894821167),
(40, 150, 40446, 1967, 0.04761904761904745, 81.67897009849548),
(40, 200, 43470, 1442, 0.024390243902438852, 90.34693884849548),
(40, 250, 52722, 1875, 0.024390243902439372, 233.7646291255951),
(40, 300, 57532, 1589, 0.047619047619047616, 173.83054327964783),
(50, 100, 37152, 1770, 0.0196078431372549, 104.80491781234741),
(50, 150, 44370, 1790, 0.019607843137254763, 93.35976600646973),
(50, 200, 53352, 2103, 0.019607843137254763, 137.70000004768372),
(50, 250, 67050, 1862, 0.0196078431372549, 194.7685990333557),
(50, 300, 86436, 2141, 0.038461538461538325, 989.3735358715057),
(75, 100, 61828, 3056, 0.038461538461538644, 294.2641839981079),
(75, 150, 91908, 3457, 0.025974025974025976, 694.4024710655212),
(75, 200, 100295, 3150, 0.038461538461538276, 422.27971291542053),
(75, 250, 115412, 2769, 0.038461538461538464, 404.5775480270386),
(75, 300, 139400, 3364, 0.038461538461538464, 1209.6577620506287),
(100, 100, 89720, 4525, 0.02912621359223301, 22.25978922843933),
(100, 150, 130181, 3952, 0.0, 328.60379791259766),
(100, 200, 141474, 3948, 0.01960784313725504, 966.167060136795),
(100, 250, 189464, 4393, 0.00990099009900976, 3673.9212460517883),
(100, 300, 201590, 4338, 0.019607843137254763, 1381.0980608463287),
(125, 100, 112150, 4535, 0.007936507936507936, 1569.5884470939636),
(125, 150, 154569, 4680, 0.007936507936507936, 523.422080039978),
(125, 200, 190950, 5166, 0.023437500000000333, 4488.701567173004),
(125, 250, 236808, 5239, 0.031007751937984607, 4525.383026838303),
(125, 300, 257300, 5245, 0.031007751937984496, 1660.6960179805756),
(150, 100, 141350, 5703, 0.038461538461538464, 34.727739095687866),
(150, 150, 202020, 6280, 0.013157894736842105, 1608.0012121200562),
(150, 200, 221040, 5605, 0.0196078431372549, 1603.6933529376984),
(150, 250, 313403, 6306, 0.013157894736841919, 1482.9248020648956),
(150, 300, 342042, 6384, 0.01960784313725509, 2934.4970400333405)]
for i in a:
    binCount, capacity, varCBC, conCBC, gapCBC, t = i
    times.update({(binCount, capacity, varCBC, conCBC, gapCBC) : (t)}) 

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection='3d')
xdata = []
ydata = []
zdata = []
for i in times.keys():
    xdata.append(i[0])
    ydata.append(i[1])
    zdata.append(times[i])
ax.scatter3D(xdata, ydata, zdata, cmap='Greens');
ax.set_xlabel('# of Bins')
ax.set_ylabel('Bin Capacity')
ax.set_zlabel('Run time (s)')
ax.view_init(20, -80)

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection='3d')
xdata = []
ydata = []
zdata = []
for i in times.keys():
    xdata.append(i[2])
    ydata.append(i[3])
    zdata.append(times[i])
ax.scatter3D(xdata, ydata, zdata, cmap='Greens');
ax.set_xlabel('# of Variables')
ax.set_ylabel('# of Constraints')
ax.set_zlabel('Run time (s)')
ax.view_init(0, 45)

### Early Termination

In [ ]:
def Test(mean,std,size,volume,solver,t):
    val = PackBins(PerfectBinPacking(mean,std,size,volume),volume,solver,t)
    dif = val - size
    return val, dif

In [ ]:
results = {}
for volume in [100,150,200,250,300]:
    for t in [1,2,3,4,5,10,15,20,25,30,60]:
        val, dif = Test(50,15,20,volume,OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING,t)
        print(val, dif)
        results.update({(volume,t) : (val, dif)})

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection='3d')
xdata = []
ydata = []
zdata = []
for i in results.keys():
    xdata.append(i[0])
    ydata.append(i[1])
    zdata.append(results[i][1])
ax.scatter3D(xdata, ydata, zdata, cmap='Greens');
ax.set_xlabel('Volume')
ax.set_ylabel('Time Limit (s)')
ax.set_zlabel('Diff. from Opt')

### TEST

In [ ]:
bin1 = [8, 4, 14, 3, 5, 6, 2, 1, 1, 7, 3, 6, 8, 13, 2, 15, 2]
bin2 = [2, 2, 20, 10, 1, 2, 18, 1, 6, 19, 19]
bin3 = [1, 1, 1, 4, 1, 4, 1, 1, 1, 2, 1, 2, 4, 1, 2, 1, 2, 2, 4, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 3, 1, 2, 1, 5, 6, 1, 2, 1, 1, 1, 1, 2, 2, 3, 2, 1, 4, 1, 1, 1, 1, 3, 2, 3]
bin4 = [2, 1, 1, 6, 5, 7, 2, 3, 1, 8, 1, 3, 1, 4, 6, 5, 3, 4, 3, 11, 4, 2, 12, 1, 2, 2]
bin5 = [3, 14, 7, 11, 1, 5, 1, 3, 6, 1, 1, 10, 1, 5, 3, 1, 8, 3, 3, 1, 2, 7, 3]
[3, 14, 7, 11, 1, 5, 1, 3, 6, 1, 1, 10, 1, 5, 3, 1, 8, 3, 3, 1, 2, 7, 3]
pack = {}
ITEMS = []
i = 0
for bin in [bin1,bin2,bin3,bin4,bin5]:
    pack.update({i : bin})
    i += 1
    for item in bin:
        ITEMS.append(item)
VOLUMES = []
amt = {}
for i in ITEMS:
    if i in VOLUMES:
        amt[i] += 1
    else:
        VOLUMES.append(i)
        amt.update({i:1})
PackBins(VOLUMES,amt,100,'GUROBI')